# Preparation

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
import langchain as lc

import warnings
warnings.filterwarnings('ignore')

load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [2]:
# Loading our vectorDB

import pickle
with open("vectorstore.pkl", "rb") as f:
    vectorstore = pickle.load(f)

# Agents

Agents use an LLM to determine which actions to perform and in what order. An action can be either using a tool and observing its output or returning it to the user. To use an agent, in addition to the concept of an LLM, it is important to understand a new concept and that of a "tool".

### 1.1 Tools
Tools are functions that agents can use to interact with the world. These tools can be common utilities (e.g. search), other chains, or even other agents.

In [15]:
llm = lc.llms.OpenAI(temperature=0.5)

tool_names = ['llm-math'] # Let use math tool
tools = lc.agents.load_tools(tool_names, llm=llm)
tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False, func=<bound method Chain.run of LLMMathChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, prompt=PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numex

In [23]:
# You can see LangChain.Agent create prompt for us. we don't need
# to create prompt again and again for same task

tool_list = [lc.agents.Tool(name='Math Tool',
             func = tools[0].run, # using above tools first index
             description="Tool to calculate, nothing else")
             ]

In [24]:
agent = lc.agents.initialize_agent(tool_list,
                                   llm,
                                   agent=lc.agents.AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                                   verbose=True)
agent.run("How are you?")



> Entering new  chain...
 This doesn't seem like a math question.
Action: None
Action Input: None
Observation: None is not a valid tool, try another one.
Thought: This doesn't seem like a math question, so I should not use the Math Tool.
Final Answer: I'm doing well, thanks for asking!

> Finished chain.


"I'm doing well, thanks for asking!"

In [25]:
agent.run("What is 100 devided by 25?")



> Entering new  chain...
 I need to use a tool to calculate this
Action: Math Tool
Action Input: 100/25
Observation: Q: Do you have any specials or promotions running currently?
A: Yes, we have a Happy Hour promotion from 3 to 5 p.m. on weekdays, with special prices on selected appetizers and drinks.
Q: What are the hours of operation for your restaurant?
A: Our restaurant is open from 11 a.m. to 10 p.m. from Monday to Saturday. On Sundays, we open at 12 p.m. and close at 9 p.m.

Q: What type of cuisine does your restaurant serve?
A: Our restaurant specializes in contemporary American cuisine with an emphasis on local and sustainable ingredients.
Q: What steps is your restaurant taking to ensure safety amid the ongoing pandemic?
A: We adhere to strict health and safety protocols, including regular sanitization, maintaining physical distance between tables, and providing hand sanitizers for customers. We also offer contactless pickup and delivery options.
Q: Do you offer vegetarian or

'4'

### 1.2 Custom Tools
You can also create your own tools by creating a class that inherits from BaseTool.

In [20]:
from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun

class CustomSearchTool(BaseTool):
    name = "restaurant search"
    description = "useful for when you need to answer questions about our restaurant"

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        store = vectorstore.as_retriever()
        docs = store.get_relevant_documents(query)
        text_list = [doc.page_content for doc in docs]
        return "\n".join(text_list)
    
    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [21]:
tools = [CustomSearchTool()]
agent = lc.agents.initialize_agent(tools, llm, agent=lc.agents.AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [22]:
agent.run("When does the restaurant open?")



> Entering new  chain...
 I need to find out when the restaurant opens
Action: restaurant search
Action Input: opening hours
Observation: Q: What are the hours of operation for your restaurant?
A: Our restaurant is open from 11 a.m. to 10 p.m. from Monday to Saturday. On Sundays, we open at 12 p.m. and close at 9 p.m.

Q: What type of cuisine does your restaurant serve?
A: Our restaurant specializes in contemporary American cuisine with an emphasis on local and sustainable ingredients.
Q: Do you have any specials or promotions running currently?
A: Yes, we have a Happy Hour promotion from 3 to 5 p.m. on weekdays, with special prices on selected appetizers and drinks.
Q: Do you offer vegetarian or vegan options?
A: Yes, we have a range of dishes to cater to vegetarians and vegans. Please let our staff know about any dietary restrictions you have when you order.
Q: What steps is your restaurant taking to ensure safety amid the ongoing pandemic?
A: We adhere to strict health and safety 

'Our restaurant is open from 11 a.m. to 10 p.m. from Monday to Saturday, and from 12 p.m. to 9 p.m. on Sundays.'